# Linear equation system solving techniques comparison



In [22]:
import numpy as np
import math
import datetime as dt
import matplotlib as mpl


<h4> 

    <i>
        index = 172110 </br>
        c = 1
        </br>
        d = 0
        </br>
        e = 1 
        </br>
        f = 2
        </br>
        9cd = 910
    </i>
</h4>

**Exercise A**


In [23]:
N = 300
iter_limit = 100
epsilon = 10.0 ** (-6)
f = 2
e = 1
a1 = a2 = a3 = 0

def init_a_exercise():
    global a1, a2, a3, iter_limit
    a1 = 5 + e
    a2 = a3 = -1
    iter_limit = 5000



**Striped matrix generating function**


In [24]:
def generate_striped_matrix(size: int):
    array = np.ndarray(shape=(size,size))
    # wybór elementu na diagonali (i,i)
    for i in range(size):
        # wybór 2 elementy przed, diagonal i 2 elementy za diagonalą
        for j in range(size):
            if 0 <= j < size:
                if abs(i - j) == 2:
                    array[i][j] = a3
                elif abs(i - j) == 1:
                    array[i][j] = a2
                elif i == j:
                    array[i][j] = a1
                else:
                    array[i][j] = 0
    return array
                



**Residue vector second norm calculation**


In [25]:
def calculate_residue_vector_norm(A: np.ndarray, x: np.ndarray, B: np.ndarray):
    n_size = A.shape[0]
    residue_norm = 0.0
    for i in range(n_size):
        row = 0.0
        for j in range(n_size):
            row += A[i][j] * x[j]
        row -= B[i]
        residue_norm += row ** 2
    return math.sqrt(residue_norm)
    



**Timing functions**

In [26]:
starting_time: dt.datetime
def tic():
    global starting_time
    starting_time = dt.datetime.now()
    
def toc() -> dt.timedelta:
    time_diff = dt.datetime.now() - starting_time 
    return time_diff


**Function printing details about tested solution.**


In [27]:
def print_details(A: np.ndarray, B: np.ndarray, x: np.ndarray, iteration_count: int, time: dt.timedelta):
    print("Residue vector second norm value: ", calculate_residue_vector_norm(A, x, B))
    print("Minutes: ", int(time.seconds / 60), "seconds: ", time.seconds % 60)
    print("Time per iteration in seconds: ", time.seconds / iteration_count)



**Matrix initialization**


In [28]:
B: np.ndarray
A: np.ndarray
x: np.ndarray

def generate_equation_system():
    global A
    global B   
    global x
    
    # Wektor pobudzenia
    B = np.ndarray(shape=(N, 1), dtype=float)
    for i in range(N):
        B[i] = math.sin(i * (f + 1))
    
    # Macierz systemowa
    A = generate_striped_matrix(N)
    
    # Macierz rozwiązań przybliżonych. Zawiera początkowe przybliżenie a później przyjmuje wartości nadane przez
    # metody iteracyjne.
    x = np.ndarray(shape=(N, 1), dtype=float)
    for i in range(N):
        x[i] = 0




# Exercise B

**Jacobi's method**

In [29]:
def jacobis_next_iteration(matrix: np.ndarray, b_vec: np.ndarray, estimates: np.ndarray) -> np.ndarray:
    this_iteration: np.ndarray = np.ndarray(shape=(estimates.shape[0],1))
    for i in range(matrix.shape[0]):
        # Copy of the i-th equation coefficients
        curr_equation = matrix[i].copy()
        # The coefficient of the x which we're currently we're estimating new value for.
        curr_calc_coeff = curr_equation[i]
        # Inserting the b vector value into the equation.
        new_x = - b_vec[i]
        for j in range(curr_equation.shape[0]):
            # for every variable except the one we're currently calculating -> (a1 * x1 + ... + an * xn - b)/ aj = xj 
            if j != i:
                # eq[j] is the coefficient of x(j) and estimates[j] is the previous value of the x(j)
                new_x += curr_equation[j] * estimates[j]
        this_iteration[i] = new_x / - curr_calc_coeff
    return this_iteration





**Gauss-Seidl's method.**


In [30]:
def gauss_seidl_next_iteration(matrix: np.ndarray, b_vec: np.ndarray, estimates: np.ndarray) -> np.ndarray:
    this_iteration: np.ndarray = np.ndarray(shape=(estimates.shape[0],1))
    for i in range(matrix.shape[0]):
        # Copy of the i-th equation coefficients
        current_equation = matrix[i].copy()
        # The coefficient of the x which we're calculating new value for
        curr_calc_coeff = current_equation[i]
        # Inserting the b vector value into the equation
        new_x = - b_vec[i]
        for j in range(i):
            new_x += current_equation[j] * this_iteration[j]
        for j in range(i + 1, matrix.shape[0]):
            new_x += current_equation[j] * estimates[j]
        this_iteration[i] = new_x / - curr_calc_coeff
    return this_iteration




    

**Testing the Jacobi's method on data from exercise A**


In [31]:
init_a_exercise()
generate_equation_system()
tic()

residue_vector_norm = calculate_residue_vector_norm(A, x, B)
i = 0
while residue_vector_norm > epsilon and i < iter_limit:
    x = jacobis_next_iteration(A, B, x)
    i += 1
    residue_vector_norm = calculate_residue_vector_norm(A, x, B)
    print("Current iteration: ", i)

# Printing details about this method time, iteration count etc.
print_details(A, B, x, i, toc())



Current iteration:  1
Current iteration:  2
Current iteration:  3
Current iteration:  4
Current iteration:  5
Current iteration:  6
Current iteration:  7
Current iteration:  8
Current iteration:  9
Current iteration:  10
Current iteration:  11
Current iteration:  12
Current iteration:  13
Current iteration:  14
Current iteration:  15
Current iteration:  16
Current iteration:  17
Current iteration:  18
Current iteration:  19
Current iteration:  20
Current iteration:  21
Current iteration:  22
Current iteration:  23
Current iteration:  24
Residue vector second norm value:  8.140195485706073e-07
Minutes:  0 seconds:  12
Time per iteration in seconds:  0.5


**Testing Gauss-Seidl's on data from A exercise**


In [32]:
init_a_exercise()
generate_equation_system()
tic()


# residue_vector_norm_values = [float]
residue_vector_norm = calculate_residue_vector_norm(A, x, B)
i = 0
while  residue_vector_norm > epsilon and i < iter_limit:
    x = gauss_seidl_next_iteration(A, B, x)
    i += 1
    residue_vector_norm = calculate_residue_vector_norm(A, x, B)
    # residue_vector_norm_values.append(residue_vector_norm)
    print("Iterations passed: ", i)
    
# Printing details about this method time, iteration count etc.
print_details(A, B, x, i, toc())




Iterations passed:  1
Iterations passed:  2
Iterations passed:  3
Iterations passed:  4
Iterations passed:  5
Iterations passed:  6
Iterations passed:  7
Iterations passed:  8
Iterations passed:  9
Iterations passed:  10
Iterations passed:  11
Iterations passed:  12
Iterations passed:  13
Iterations passed:  14
Iterations passed:  15
Iterations passed:  16
Residue vector second norm value:  7.696797973615508e-07
Minutes:  0 seconds:  8
Time per iteration in seconds:  0.5


## Exercise C

**Changing the coefficients used to generate equation system to match the specification of exercise C**

In [33]:

def init_c_exercise():
    global a1, a2, a3, iter_limit
    a1 = 3
    a2 = a3 = -1
    iter_limit = 30


**Testing the Jacobi's method on data from exercise C**


In [34]:
init_c_exercise()
generate_equation_system()
tic()

i = 0
residue_vector_norm = calculate_residue_vector_norm(A, x, B)
while residue_vector_norm > epsilon and i < iter_limit:
    i += 1
    x = jacobis_next_iteration(A, B, x)
    residue_vector_norm = calculate_residue_vector_norm(A, x, B)
    print("Iterations passed: ", i)

# Printing details about this method time, iteration count etc.
print_details(A, B, x, i, toc())



Iterations passed:  1
Iterations passed:  2
Iterations passed:  3
Iterations passed:  4
Iterations passed:  5
Iterations passed:  6
Iterations passed:  7
Iterations passed:  8
Iterations passed:  9
Iterations passed:  10
Iterations passed:  11
Iterations passed:  12
Iterations passed:  13
Iterations passed:  14
Iterations passed:  15
Iterations passed:  16
Iterations passed:  17
Iterations passed:  18
Iterations passed:  19
Iterations passed:  20
Iterations passed:  21
Iterations passed:  22
Iterations passed:  23
Iterations passed:  24
Iterations passed:  25
Iterations passed:  26
Iterations passed:  27
Iterations passed:  28
Iterations passed:  29
Iterations passed:  30
Iterations passed:  31
Iterations passed:  32
Iterations passed:  33
Iterations passed:  34
Iterations passed:  35
Iterations passed:  36
Iterations passed:  37
Iterations passed:  38
Iterations passed:  39
Iterations passed:  40
Iterations passed:  41
Iterations passed:  42
Iterations passed:  43
Iterations passed:  

**Testing Gauss-Seidl's on data from C exercise**


In [26]:
init_c_exercise()
generate_equation_system()
tic()

i = 0
residue_vector_norm_values = [float]
residue_vector_norm = calculate_residue_vector_norm(A, x, B)  
while  residue_vector_norm > epsilon and i < iter_limit:
    i += 1
    x = gauss_seidl_next_iteration(A, B, x)
    residue_vector_norm = calculate_residue_vector_norm(A, x, B)
    residue_vector_norm_values.append(residue_vector_norm)
    print("Iterations passed: ", i)
    
# Printing details about this method time, iteration count etc.
print_details(A, B, x, i, toc())



Iterations passed:  1
Iterations passed:  2
Iterations passed:  3
Iterations passed:  4
Iterations passed:  5
Iterations passed:  6
Iterations passed:  7
Iterations passed:  8
Iterations passed:  9
Iterations passed:  10
Iterations passed:  11
Iterations passed:  12
Iterations passed:  13
Iterations passed:  14
Iterations passed:  15
Iterations passed:  16
Iterations passed:  17
Iterations passed:  18
Iterations passed:  19
Iterations passed:  20
Iterations passed:  21
Iterations passed:  22
Iterations passed:  23
Iterations passed:  24
Iterations passed:  25
Iterations passed:  26
Iterations passed:  27
Iterations passed:  28
Iterations passed:  29
Iterations passed:  30
Iterations passed:  31
Iterations passed:  32
Iterations passed:  33
Iterations passed:  34
Iterations passed:  35
Iterations passed:  36
Iterations passed:  37
Iterations passed:  38
Iterations passed:  39
Iterations passed:  40
Iterations passed:  41
Iterations passed:  42
Iterations passed:  43
Iterations passed:  

## Exercise C

**LU Factorization method**

*LU Decomposition*

In [27]:

def decompose_matrix_in_place(A: np.ndarray):
    # For every element on the main diagonal (the matrix has to be of shape NxN)
    for k in range(A.shape[0]):
        # Column normalization
        for i in range(k+1, A.shape[0]):
            if math.fabs(A[k][k]) >= epsilon:
                A[i][k] = A[i][k] / A[k][k]
            else:
                print("An element too close to zero was placed on the diagonal")
                return  # the element on the main diagonal is too close to zero to continue operations
        # Submatrix normalization
        for i in range(k + 1, A.shape[0]):
            for j in range(k + 1, A.shape[0]):
                # Skipping the main diagonal
                if i != j:
                    A[i][j] -= A[i][k] * A[k][j]



**Solving the decomposed system**


In [28]:
# this returns our theoretical z
def solve_L(LU: np.ndarray, b: np.ndarray) -> np.ndarray:
    sol = np.zeros(shape=(LU.shape[0], 1), dtype=float)
    # We assume that all elements of the main diagonal of the L matrix are = 1
    sol[0][0] = b[0][0]
    for i in range(1, LU.shape[0]):
        for j in range(0, i):
            sol[i][0] += sol[j][0] * LU[i][j]
        sol[i][0] -= b[i][0]
    return sol
        
# this returns our x
def solve_U(LU: np.ndarray, z: np.ndarray) -> np.ndarray:
    n = LU.shape[0]
    
    sol = np.zeros(shape=(n, 1), dtype=float)
    # n - 1 is the last index of matrix => last x element = last b / last element on diagonal
    sol[n - 1][0] = z[n - 1][0] / LU[n - 1][n - 1]
    
    for i in reversed(range(0, n)):
        for j in reversed(range(n - 1, i)):
            sol[i][0] += sol[j][0] * LU[i][j]
        sol[i][0] -= z[i][0]
        sol[i][0] /= LU[i][i]
    return sol
    

def solve_LU(LU: np.ndarray, b: np.ndarray):
    z = solve_L(LU, b)
    x = solve_U(LU, z)
    return x
    



## One more attempt to LU Fac.


In [29]:
def matrix_decomposition(A: np.ndarray) -> (np.ndarray, np.ndarray):
    n = A.shape[0]
    L = np.ndarray(shape=(n, n))
    U = np.ndarray(shape=(n, n))
    # decomposition
    for i in range(n):
        # upper triangular
        for k in range(i,n):
            lu_sum = 0.0
            for j in range(0, i):
                lu_sum +=  L[i][j] * U[j][k]
            U[i][j] = A[i][j] - lu_sum
                
        for k in range(i, n):
            if i == k:
                L[i][i] = 1
                continue
            lu_sum = 0.0
            for j in range(0, i):
                lu_sum += U[k][j] * L[j][i]
            L[k][i] = (A[k][i] - sum) / U[i][i]

    
    
def solve_lu_equation(L: np.ndarray, U: np.ndarray, B: np.ndarray):
    n = L.shape[0]
    L_inv = np.ndarray(shape=L.shape)
    for i in range(n):
        for j in range(i):
            L_inv[i][j] = -L[i][j]
        L_inv[i][i] = L[i][i]
    y = np.dot



**LU Factorization method testing**
            

In [30]:
init_c_exercise()
generate_equation_system()

L, U = matrix_decomposition(A)

print("Residue vector norm value: ", calculate_residue_vector_norm(A, xc, B))


UnboundLocalError: local variable 'j' referenced before assignment

Initializing last exercise



In [ ]:


print("100 x 100: ")
N =  100
init_a_exercise()
generate_equation_system()
tic()

residue_vector_norm = calculate_residue_vector_norm(A, x, B)
i = 0
while residue_vector_norm > epsilon and i < iter_limit:
    x = jacobis_next_iteration(A, B, x)
    i += 1
    residue_vector_norm = calculate_residue_vector_norm(A, x, B)
    print("Current iteration: ", i)

print_details(A, B, x, i, toc())


init_a_exercise()
generate_equation_system()
tic()

residue_vector_norm = calculate_residue_vector_norm(A, x, B)
i = 0
while residue_vector_norm > epsilon and i < iter_limit:
    x = gauss_seidl_next_iteration(A, B, x)
    i += 1
    residue_vector_norm = calculate_residue_vector_norm(A, x, B)
    print("Current iteration: ", i)

print_details(A, B, x, i, toc())




print("500 x 500")
N = 500  

init_a_exercise()
generate_equation_system()
tic()


residue_vector_norm = calculate_residue_vector_norm(A, x, B)
i = 0
while residue_vector_norm > epsilon and i < iter_limit:
    x = jacobis_next_iteration(A, B, x)
    i += 1
    residue_vector_norm = calculate_residue_vector_norm(A, x, B)
    print("Iterations passed: ", i)
    
print_details(A, B, x, i, toc())


generate_equation_system()
tic()


residue_vector_norm = calculate_residue_vector_norm(A, x, B)
i = 0
while residue_vector_norm > epsilon and i < iter_limit:
    x = gauss_seidl_next_iteration(A, B, x)
    i += 1
    residue_vector_norm = calculate_residue_vector_norm(A, x, B)
    # residue_vector_norm_values.append(residue_vector_norm)
    print("Iterations passed: ", i)
    
# Printing details about this method time, iteration count etc.
print_details(A, B, x, i, toc())








print("1000 x 1000: ")


N = 1000  

init_a_exercise()
generate_equation_system()
tic()
residue_vector_norm = calculate_residue_vector_norm(A, x, B)
i = 0
while residue_vector_norm > epsilon and i < iter_limit:
    x = jacobis_next_iteration(A, B, x)
    i += 1
    residue_vector_norm = calculate_residue_vector_norm(A, x, B)
    print("Current iteration: ", i)

# Printing details about this method time, iteration count etc.
print_details(A, B, x, i, toc())

init_a_exercise()
generate_equation_system()
tic()
residue_vector_norm = calculate_residue_vector_norm(A, x, B)
i = 0
while residue_vector_norm > epsilon and i < iter_limit:
    x = gauss_seidl_next_iteration(A, B, x)
    i += 1
    residue_vector_norm = calculate_residue_vector_norm(A, x, B)
    print("Current iteration: ", i)

# Printing details about this method time, iteration count etc.
print_details(A, B, x, i, toc())






print("2000 x 2000: ")

N = 2000  

init_a_exercise()
generate_equation_system()
tic()
# residue_vector_norm_values = [float]
residue_vector_norm = calculate_residue_vector_norm(A, x, B)
i = 0
while residue_vector_norm > epsilon and i < iter_limit:
    x = jacobis_next_iteration(A, B, x)
    i += 1
    residue_vector_norm = calculate_residue_vector_norm(A, x, B)
    # residue_vector_norm_values.append(residue_vector_norm)
    print("Iterations passed: ", i)
    
# Printing details about this method time, iteration count etc.
print_details(A, B, x, i, toc())


init_a_exercise()
generate_equation_system()
tic()
residue_vector_norm = calculate_residue_vector_norm(A, x, B)
i = 0
while residue_vector_norm > epsilon and i < iter_limit:
    x = gauss_seidl_next_iteration(A, B, x)
    i += 1
    residue_vector_norm = calculate_residue_vector_norm(A, x, B)
    # residue_vector_norm_values.append(residue_vector_norm)
    print("Iterations passed: ", i)
    
# Printing details about this method time, iteration count etc.
print_details(A, B, x, i, toc())





100 x 100: 
Current iteration:  1
Current iteration:  2
Current iteration:  3
Current iteration:  4
Current iteration:  5
Current iteration:  6
Current iteration:  7
Current iteration:  8
Current iteration:  9
Current iteration:  10
Current iteration:  11
Current iteration:  12
Current iteration:  13
Current iteration:  14
Current iteration:  15
Current iteration:  16
Current iteration:  17
Current iteration:  18
Current iteration:  19
Current iteration:  20
Current iteration:  21
Current iteration:  22
Current iteration:  23
Current iteration:  24
Residue vector second norm value:  8.124472480419429e-07
Minutes:  0 seconds:  1
Time per iteration in seconds:  0.041666666666666664
500 x 500
Iterations passed:  1
Iterations passed:  2
Iterations passed:  3
Iterations passed:  4
Iterations passed:  5
Iterations passed:  6
Iterations passed:  7
Iterations passed:  8
Iterations passed:  9
Iterations passed:  10
Iterations passed:  11
Iterations passed:  12
Iterations passed:  13
Iterations 

In [37]:
print("3000 x 3000: ")

N = 3000
init_a_exercise()
generate_equation_system()

tic()
residue_vector_norm = calculate_residue_vector_norm(A, x, B)
i = 0
while residue_vector_norm > epsilon and i < iter_limit:
    x = jacobis_next_iteration(A, B, x)
    i += 1
    residue_vector_norm = calculate_residue_vector_norm(A, x, B)
    print("Current iteration: ", i)

# Printing details about this method time, iteration count etc.
print_details(A, B, x, i, toc())

init_a_exercise()
generate_equation_system()

tic()
residue_vector_norm = calculate_residue_vector_norm(A, x, B)
i = 0
while residue_vector_norm > epsilon and i < iter_limit:
    x = gauss_seidl_next_iteration(A, B, x)
    i += 1
    residue_vector_norm = calculate_residue_vector_norm(A, x, B)
    print("Current iteration: ", i)

# Printing details about this method time, iteration count etc.
print_details(A, B, x, i, toc())


3000 x 3000: 
Current iteration:  1
Current iteration:  2
Current iteration:  3
Current iteration:  4
Current iteration:  5
Current iteration:  6
Current iteration:  7
Current iteration:  8
Current iteration:  9
Current iteration:  10
Current iteration:  11
Current iteration:  12
Current iteration:  13
Current iteration:  14
Current iteration:  15
Current iteration:  16
Current iteration:  17
Current iteration:  18
Current iteration:  19
Current iteration:  20
Current iteration:  21
Current iteration:  22
Current iteration:  23
Residue vector second norm value:  6.878638859433061e-07
Minutes:  19 seconds:  7
Time per iteration in seconds:  49.869565217391305
